In [ ]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import OrdinalEncoder
from sklearn import model_selection
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold 
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_curve,roc_auc_score
cat_encoder = OneHotEncoder( )
ordinal_encoder = OrdinalEncoder()

data = pd.read_csv("heart_2020_cleaned.csv")
data.head()
data['BMI']


In [ ]:
#plot
colors2 = sns.color_palette(['#1337f5', '#E80000'], 2)
colors1 = sns.color_palette(['#1337f5'], 1)
plt.figure(figsize=(15,7))
plt.title('HeartDisease Count') 
sns.countplot(data=data, x='HeartDisease', palette=colors2, order=data['HeartDisease'].value_counts().index)
data.hist(figsize=(16, 8), bins=50, color=colors1) 
plt.suptitle("Distribution of Numerical Values") 


obj_cols = data.select_dtypes(include='object').columns[1:]
num_cols = data.select_dtypes(exclude='object').columns
print(f'Object columns : {obj_cols}', end='\n\n')
print(f'Numberical columns : {num_cols}')
plt.figure(figsize=(20, 40))
for i in range(len(obj_cols)):
  plt.subplot(7, 2, i+1)

  if(data[obj_cols[i]].nunique() < 3):
    ax = sns.countplot(data=data, x=obj_cols[i], palette=colors2, order=data[obj_cols[i]].value_counts().index[:6])
  else:
    ax = sns.countplot(data=data, x=obj_cols[i], palette=colors2, order=data[obj_cols[i]].value_counts().index[:6])

  
  plt.title(f'{obj_cols[i]}', fontsize=15, fontweight='bold', color='brown')
  plt.subplots_adjust(hspace=0.5)

  for p in ax.patches:
    height = p.get_height() 
    width = p.get_width()
    percent = height/len(data)

    ax.text(x=p.get_x()+width/2, y=height+2, s=format(percent, ".2%"), fontsize=12, ha='center', weight='bold')

In [ ]:
#Most of people in our data are white and have no diabetic.
#Most of them had done a physical activity during the past 30 days other than their regular job and in general they have very good health as they said.
#A litle of them who have asthma, kidney disease and skin cancer.
#Most people said that they have generaly very good health. A few of people who said that they have generaly a poor health.
data.corr()


In [ ]:
#data=data.drop_duplicates()
#data.shape
print(data["Sex"].describe())

coulmn_data = data.columns
for col in coulmn_data:
    total = np.sum(data[col].isnull())
    print('{} - {}%'.format(col, round(total)))

In [ ]:
cols = data.columns
num_cols = data._get_numeric_data().columns
Categ_cols=list(set(cols) - set(num_cols))
lenght=len(Categ_cols)
from sklearn.compose import make_column_transformer
for i in range (lenght):
    print('____________________________________________________________________________________')
    #Variable
    print('Variable name:' ,Categ_cols[i])
    #Unique values 
    print('Unique values are:', data[Categ_cols[i]].unique())
    #Number of unique values
    print('Number of unique values:',data[Categ_cols[i]].nunique())
    # Frequencies of categorical variables
    FreqValue=data[Categ_cols[i]].value_counts()
    print('Frequency of each unique value in a catagorical variable')
    print(FreqValue)


In [ ]:
numerical = data.select_dtypes(exclude='object')
for col in numerical:
  
  Q1, Q3 = data[col].quantile([0.25,0.75])
  IQR = Q3 - Q1
  right = Q3 + 1.5 * IQR
  left = Q1 - 1.5 * IQR
  data[col] = np.where(data[col]>IQR, Q3, data[col])
  n = len(data.loc[(data[col] < left) | (data[col] > right)])
  print(f"{col}:\n\tOutlier Num = {n}\n\tOutlier Percentage = {n*100/len(data):.2f}%\n")
  # fit outs

In [ ]:

#ordinal encoding won't make any sence here. Will prefer values with higher numbes
'''data["Diabetic"] = data["Diabetic"].replace(
    {
        "Yes" : 1,
        "Yes (during pregnancy)" : 2,
        "No, borderline diabetes" : 3,
        "No" : 3
    }
)'''  

#data["GenHealth"] = data["GenHealth"].replace(
 #   {
  #      'Excellent' : 4,
   #     'Very good' : 3,
    #    'Good' : 2,
     #   'Fair' : 1,
      #  'Poor' : 0
    #}
#)
data[['Asthma']] = ordinal_encoder.fit_transform(data[['Asthma']])
data[['DiffWalking']] = ordinal_encoder.fit_transform(data[['DiffWalking']])
data[['PhysicalActivity']] = ordinal_encoder.fit_transform(data[['PhysicalActivity']])
data[['HeartDisease']] = ordinal_encoder.fit_transform(data[['HeartDisease']])
data[['KidneyDisease']] = ordinal_encoder.fit_transform(data[['KidneyDisease']])
data[['SkinCancer']] = ordinal_encoder.fit_transform(data[['SkinCancer']])
data[['Stroke']] = ordinal_encoder.fit_transform(data[['Stroke']])
data[['AlcoholDrinking']] = ordinal_encoder.fit_transform(data[['AlcoholDrinking']])
data[['Smoking']] = ordinal_encoder.fit_transform(data[['Smoking']])
data[['GenHealth']] = ordinal_encoder.fit_transform(data[['GenHealth']])
data[['Diabetic']] = ordinal_encoder.fit_transform(data[['Diabetic']])
data[['AgeCategory']] = ordinal_encoder.fit_transform(data[['AgeCategory']])
#data.replace("Yes",1,inplace=True)
#data.replace("No",0 ,inplace=True)
#data.replace("18-24",0,inplace=True)
#data.replace("25-29",1,inplace=True)
#data.replace("30-34",2,inplace=True)
#data.replace("35-39",3,inplace=True)
#data.replace("40-44",4,inplace=True)
#data.replace("45-49",5,inplace=True)
#data.replace("50-54",6,inplace=True)
#data.replace("55-59",7,inplace=True)
#data.replace("60-64",8,inplace=True)
#data.replace("65-69",9,inplace=True)
#data.replace("70-74",10,inplace=True)
#data.replace("75-79",11,inplace=True)
#data.replace("80 or older",12,inplace=True)

data[['Race']] = ordinal_encoder.fit_transform(data[['Race']])

#data.replace("White",0,inplace=True)
#data.replace("Other",1,inplace=True)
#data.replace("Black",2,inplace=True)
#data.replace("Hispanic",3,inplace=True)
#data.replace("Asian",4,inplace=True)
#data.replace("American Indian/Alaskan Native", 4, inplace=True)

#one hot-encoding
#data.replace("Female",0,inplace=True)

gender = pd.get_dummies(data.Sex)
data = pd.concat([data, gender], axis='columns')
data.drop('Sex', axis=1, inplace=True)




In [ ]:
#
#x_train_set, x_test, y_train_set,y_test_set = sklearn.model_selection.train_test_split(X, Y, test_size=0.2, random_state=42, stratify=Y)
#
#print(y_train_set.value_counts()/len(y_train_set))
#print(y_test_set.value_counts()/len(y_test_set))
#print(X.value_counts()/len(y_test_set))

In [ ]:
def try_model(model_pipeline, X_train, y_train, X_test, y_test):
    
    model_pipeline.fit(X_train, y_train)
    y_pred = model_pipeline.predict(X_test)
    print(model_pipeline.score(X_test, y_test))

    print(f"precision {precision_score(y_test, y_pred)}")
    print(f"recall {recall_score(y_test, y_pred)}")
    print(f"f1_score {f1_score(y_test, y_pred)}")
   
    #print(f'Training Accuracy : {model_pipeline.score(X_train, y_train)}')
    #print(f'Test Accuracy : {model_pipeline.score(X_test, y_test)}')
    #confusion_matrix(X_train, y_train)

    return y_pred, classification_report(y_test, y_pred)

In [ ]:
def plot_roc_curve(fpr, tpr, label=None):
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0,1], [0,1], 'k--')
    [...]


    

In [ ]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

In [ ]:
 
data.columns = data.columns.to_series().apply(lambda x: x.strip())
X = data.drop(columns=["HeartDisease"])

Y = data["HeartDisease"] 
print(X.shape)
print(Y.shape)
from sklearn.model_selection import StratifiedKFold
kf = StratifiedKFold(n_splits=3)
for train_index, test_index in kf.split(X, Y):
    print("TRAIN:", train_index, "TEST:", test_index)
    x_train_set, x_test=X.reindex( train_index), X.reindex( test_index) 
    y_train_set, y_test_set = Y[train_index], Y[test_index]
    print(y_train_set.value_counts()/len(y_train_set))
    print(y_test_set.value_counts()/len(y_test_set))
    print(X.value_counts()/len(y_test_set))
    lr = LogisticRegression()
    y_pred, report = try_model(lr,
                            x_train_set,
                            y_train_set,
                            x_test,
                            y_test_set)
    y_predict_prob = lr.predict_proba(x_test)[:, 1]

    print(report)
    fpr, tpr, thresholds = roc_curve(y_test_set, y_predict_prob)
    auc = roc_auc_score(y_test_set, y_predict_prob)
    sns.set_style("white")
    plt.figure(figsize = [8, 5])
    plt.plot(fpr, tpr, label = (f"AUC = {auc:.3f}"))
    plt.plot((0,1), ls = "--")
    plt.title("Logistic Regression ROC curve", size = 13)
    plt.xlabel("False Positive Rate", size = 11)
    plt.ylabel("True Positive Rate", size = 11)
    plt.legend(loc = 'best')
    plt.show()
    plt.show() 

In [ ]:
data

In [ ]:
data.plot(kind='scatter', x='HeartDisease', y='Diabetic')
corr_matrix = data.corr()
corr_matrix['HeartDisease'].sort_values(ascending=False)